In [ ]:
#prophet

In [8]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "prophet_output_models"
output_csv = "prophet_output_csv"
output_graphs = "prophet_output_graphs"
output_metrics_csv = "prophet_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to handle multiple date formats
# -----------------------------
def parse_dates(date_series):
    """Try multiple date formats and return parsed dates"""
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%m/%d/%Y"):
        try:
            return pd.to_datetime(date_series, format=fmt, errors="coerce")
        except:
            continue
    # fallback: let pandas auto-infer
    return pd.to_datetime(date_series, errors="coerce")

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"\n========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # Parse dates
        df['Date'] = parse_dates(df['Date'])
        if df['Date'].isna().sum() > 0:
            print(f"⚠️ Warning: {df['Date'].isna().sum()} invalid dates found and dropped")
            df = df.dropna(subset=['Date'])

        df = df.sort_values('Date')

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'] = df['Average Price'].fillna(mean_value)
            print(f"Filled missing values with mean: {mean_value:.2f}")

        # Prepare data for Prophet
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # Build and fit Prophet model
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        # Predict for existing dates
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        df['Predicted'] = forecast['yhat'].values

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Save CSV with Actual vs Predicted
        output_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_results.csv"))
        output_df.to_csv(updated_csv_path, index=False)

        # Calculate metrics
        y_true = df['Average Price'].values
        y_pred = df['Predicted'].values

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        # Print metrics
        print(f"Metrics for {file}:")
        print(f"  MAE      : {mae}")
        print(f"  RMSE     : {rmse}")
        print(f"  R²       : {r2}")
        print(f"  MAPE     : {mape}%")
        print(f"  Accuracy : {accuracy}%")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_model.pkl"))
        joblib.dump(model, model_file)

        # Save Graph
        plt.figure(figsize=(10, 6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue', linewidth=2)
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linewidth=2, linestyle='--')
        plt.title(f"Actual vs Predicted - {file}", fontsize=14)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Price", fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()

        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_graph.png"))
        plt.savefig(graph_path)
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"\n✅ All districts processed! Metrics saved to {output_metrics_csv}, CSV results saved, and graphs generated.")


========== Processing: capsicum_Bangalore_weekly.csv ==========


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\bef4kqsb.json
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\j3t4iysk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\ravik\\AppData\\Roaming\\Python\\Python312\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=69320', 'data', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\bef4kqsb.json', 'init=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\j3t4iysk.json', 'output', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\prophet_modelzxcht4hs\\prophet_model-20251119175452.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:54:52 - cmdstanpy - INFO - Chain [1] s

Metrics for capsicum_Bangalore_weekly.csv:
  MAE      : 544.43
  RMSE     : 749.5
  R²       : 0.45
  MAPE     : 21.5%
  Accuracy : 78.5%


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\pjpz0uyu.json
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\kutvteaq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\ravik\\AppData\\Roaming\\Python\\Python312\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=57650', 'data', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\pjpz0uyu.json', 'init=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\kutvteaq.json', 'output', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\prophet_modelqg1zuy1h\\prophet_model-20251119175454.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:54:54 - cmdstanpy - INFO - Chain [1] s


========== Processing: capsicum_Belgaum_weekly.csv ==========


...

Metrics for capsicum_Mysore_weekly.csv:
  MAE      : 457.65
  RMSE     : 590.53
  R²       : 0.69
  MAPE     : 18.41%
  Accuracy : 81.59%


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\31zbq43f.json
DEBUG:cmdstanpy:input tempfile: C:\Users\ravik\AppData\Local\Temp\tmpieyd2j_3\zguoe532.json



========== Processing: capsicum_Udupi_weekly.csv ==========


DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\ravik\\AppData\\Roaming\\Python\\Python312\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=21678', 'data', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\31zbq43f.json', 'init=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\zguoe532.json', 'output', 'file=C:\\Users\\ravik\\AppData\\Local\\Temp\\tmpieyd2j_3\\prophet_modely5nr6sp6\\prophet_model-20251119175500.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:55:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:55:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Metrics for capsicum_Udupi_weekly.csv:
  MAE      : 587.97
  RMSE     : 794.79
  R²       : 0.71
  MAPE     : 17.69%
  Accuracy : 82.31%

✅ All districts processed! Metrics saved to prophet_metrics.csv, CSV results saved, and graphs generated.


In [ ]:
#ARIMA

In [10]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# Create output directories
# -----------------------------
input_folder = "dataset"
output_models = "arima_output_models"
output_csv = "arima_output_csv"
output_graphs = "arima_output_graphs"
output_logs = "arima_output_logs"
output_metrics = "arima_output_metrics"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)
os.makedirs(output_metrics, exist_ok=True)

# Metrics collector
all_metrics = []

# -----------------------------
# Helper function for flexible date parsing
# -----------------------------
def parse_dates(date_series):
    """Try multiple date formats until successful."""
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%Y/%m/%d"):
        try:
            return pd.to_datetime(date_series, format=fmt)
        except Exception:
            continue
    return pd.to_datetime(date_series, errors="coerce")  # fallback

# -----------------------------
# Process each CSV file
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"Processing: {file}")

        try:
            # Load CSV
            df = pd.read_csv(file_path)

            # Handle multiple date formats
            df['Date'] = parse_dates(df['Date'])
            df = df.dropna(subset=['Date'])
            df = df.sort_values('Date')
            df.set_index('Date', inplace=True)

            # Handle NaN values in Average Price
            df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

            # Add moving averages
            df['MA_7'] = df['Average Price'].rolling(window=7).mean()
            df['MA_30'] = df['Average Price'].rolling(window=30).mean()

            # ARIMA parameters (static, can be tuned later)
            p, d, q = 5, 1, 0
            model = ARIMA(df['Average Price'], order=(p, d, q))
            model_fit = model.fit()

            # Save ARIMA model
            model_file = os.path.join(output_models, file.replace(".csv", "_arima_model.pkl"))
            joblib.dump(model_fit, model_file)

            # Predictions
            predictions = model_fit.predict(start=0, end=len(df)-1, typ='levels')
            df['Predicted'] = predictions

            # Round Actual and Predicted to 2 decimals
            df['Average Price'] = df['Average Price'].round(2)
            df['Predicted'] = df['Predicted'].round(2)

            # Save Actual vs Predicted CSV
            comparison_df = pd.DataFrame({
                "Date": df.index,
                "Actual": df['Average Price'],
                "Predicted": df['Predicted']
            })
            comparison_csv = os.path.join(output_csv, file.replace(".csv", "_arima_actual_vs_predicted.csv"))
            comparison_df.to_csv(comparison_csv, index=False)

            # Calculate metrics
            mae = round(mean_absolute_error(df['Average Price'], df['Predicted']), 2)
            rmse = round(np.sqrt(mean_squared_error(df['Average Price'], df['Predicted'])), 2)
            r2 = round(r2_score(df['Average Price'], df['Predicted']), 2)
            mape = round(np.mean(np.abs((df['Average Price'] - df['Predicted']) / df['Average Price'])) * 100, 2)
            accuracy = round(100 - mape, 2)

            all_metrics.append({
                "District": file.replace(".csv", ""),
                "MAE": mae,
                "RMSE": rmse,
                "R2": r2,
                "MAPE(%)": mape,
                "Accuracy (%)": accuracy
            })

            # Print metrics
            print(f"Metrics for {file}:")
            print(f"  MAE      : {mae}")
            print(f"  RMSE     : {rmse}")
            print(f"  R²       : {r2}")
            print(f"  MAPE     : {mape}%")
            print(f"  Accuracy : {accuracy}%\n")

            # Save graph
            plt.figure(figsize=(12,6))
            plt.plot(df.index, df['Average Price'], label="Actual", color="blue", linewidth=2)
            plt.plot(df.index, df['Predicted'], label="Predicted (ARIMA)", color="red", linestyle="dashed", linewidth=2)
            plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
            plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
            plt.xlabel("Date")
            plt.ylabel("Average Price")
            plt.title(f"Price Prediction (ARIMA) - {file}")
            plt.legend()
            plt.grid(True)
            graph_file = os.path.join(output_graphs, file.replace(".csv", "_arima_graph.png"))
            plt.savefig(graph_file)
            plt.close()

            # Save logs
            log_file = os.path.join(output_logs, file.replace(".csv", "_arima_training.txt"))
            with open(log_file, "w", encoding="utf-8") as f:
                f.write(f"ARIMA model fitted for {file}.\n")
                f.write(f"ARIMA Order: (p={p}, d={d}, q={q})\n")
                f.write(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}, MAPE: {mape}%, Accuracy: {accuracy}%\n")
                f.write("Predictions, Graph, Logs saved.\n")

            print(f"Done with {file} | Model, CSV, Graph, Metrics, Log saved.\n")

        except Exception as e:
            print(f"Error processing {file}: {e}")

# -----------------------------
# Save all metrics in one CSV
# -----------------------------
if all_metrics:
    metrics_df = pd.DataFrame(all_metrics)
    metrics_csv = os.path.join(output_metrics, "arima_all_districts_metrics.csv")
    metrics_df.to_csv(metrics_csv, index=False)
    print(f"All metrics saved to {metrics_csv}")

Processing: capsicum_Bangalore_weekly.csv
Metrics for capsicum_Bangalore_weekly.csv:
  MAE      : 243.02
  RMSE     : 361.25
  R²       : 0.87
  MAPE     : 8.55%
  Accuracy : 91.45%

Done with capsicum_Bangalore_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: capsicum_Belgaum_weekly.csv
Metrics for capsicum_Belgaum_weekly.csv:
  MAE      : 267.22
  RMSE     : 459.48
  R²       : 0.62
  MAPE     : 8.62%
  Accuracy : 91.38%

Done with capsicum_Belgaum_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: capsicum_Chikmagalur_weekly.csv
Metrics for capsicum_Chikmagalur_weekly.csv:
  MAE      : 69.44
  RMSE     : 216.6
  R²       : 0.87
  MAPE     : 3.44%
  Accuracy : 96.56%

Done with capsicum_Chikmagalur_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: capsicum_Davangere_weekly.csv
Metrics for capsicum_Davangere_weekly.csv:
  MAE      : 328.36
  RMSE     : 601.24
  R²       : 0.88
  MAPE     : 9.91%
  Accuracy : 90.09%

Done with capsicum_Davan

In [ ]:
#SARIMAX

In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")  # Ignore warnings

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "sarimax_output_models"
output_csv = "sarimax_output_csv"
output_graphs = "sarimax_output_graphs"
output_metrics_csv = "sarimax_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])  # Drop rows where date conversion failed
        df = df.sort_values('Date')
        df.set_index('Date', inplace=True)

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled missing values with mean: {mean_value:.2f}")

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # SARIMAX order (tune if needed)
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, 7)

        # Fit SARIMAX model
        model = SARIMAX(df['Average Price'],
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Predictions
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Calculate metrics
        y_true = df['Average Price'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        # Display metrics
        print(f"Metrics for {file}:")
        print(f"  MAE      : {mae}")
        print(f"  RMSE     : {rmse}")
        print(f"  R²       : {r2}")
        print(f"  MAPE     : {mape}%")
        print(f"  Accuracy : {accuracy}%\n")

        # Save metrics to list
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save Actual vs Predicted CSV
        actual_vs_pred = df[['Average Price', 'Predicted']].reset_index()
        actual_vs_pred.rename(columns={'Average Price': 'Actual'}, inplace=True)
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_actual_vs_predicted.csv"))
        actual_vs_pred.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['Average Price'], label="Actual", color="blue")
        plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
        plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
        plt.plot(df.index, df['Predicted'], label="Predicted (SARIMAX)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"Price Prediction (SARIMAX) - {file}")
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_sarimax_graph.png"))
        plt.savefig(graph_file)
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)

print(f"✅ All districts processed! Metrics saved to {output_metrics_csv}, predictions CSVs, graphs, and models generated.")

========== Processing: capsicum_Bangalore_weekly.csv ==========
Metrics for capsicum_Bangalore_weekly.csv:
  MAE      : 249.43
  RMSE     : 368.89
  R²       : 0.87
  MAPE     : 8.84%
  Accuracy : 91.16%

========== Processing: capsicum_Belgaum_weekly.csv ==========
Metrics for capsicum_Belgaum_weekly.csv:
  MAE      : 319.81
  RMSE     : 492.72
  R²       : 0.56
  MAPE     : 10.41%
  Accuracy : 89.59%

========== Processing: capsicum_Chikmagalur_weekly.csv ==========
Metrics for capsicum_Chikmagalur_weekly.csv:
  MAE      : 99.08
  RMSE     : 271.36
  R²       : 0.79
  MAPE     : 4.89%
  Accuracy : 95.11%

========== Processing: capsicum_Davangere_weekly.csv ==========
Metrics for capsicum_Davangere_weekly.csv:
  MAE      : 341.56
  RMSE     : 616.53
  R²       : 0.87
  MAPE     : 10.8%
  Accuracy : 89.2%

========== Processing: capsicum_Hassan_weekly.csv ==========
Metrics for capsicum_Hassan_weekly.csv:
  MAE      : 68.25
  RMSE     : 154.0
  R²       : 0.97
  MAPE     : 3.91%
  Acc

In [ ]:
#TAT + MHA

In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Define Temporal Attention Transformer Model
# -----------------------------
def build_tat_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)  # (look_back, 1)
    
    # Multi-Head Attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual connection
    
    # Feed-Forward Network
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual connection
    
    # Global Pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # Output
    outputs = layers.Dense(1)(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)
        
        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue
        
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Replace NaN in 'Average Price' with column mean
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # Replace NaN in moving averages with column mean
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data using MinMaxScaler
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        # Create sequences
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Compute metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_model.h5"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save all metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_weekly.csv


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)    │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_5        │ (None, 30, 1)             │           1,793 │ input_layer_5[0][0],       │
│ (MultiHeadAttention)          │                           │                 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_11 (Dropout)          │ (None, 30, 1)             │               0 │ multi_head_attention_5[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_10 (Add)                  │ (None, 30, 1)             │               0 │ dropout_11[0][0],          │
│                               │                           │                 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_10        │ (None, 30, 1)             │               2 │ add_10[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_15 (Dense)              │ (None, 30, 128)           │             256 │ layer_normalization_10[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_16 (Dense)              │ (None, 30, 1)             │             129 │ dense_15[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_11 (Add)                  │ (None, 30, 1)             │               0 │ layer_normalization_10[0]… │
│                               │                           │                 │ dense_16[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_11        │ (None, 30, 1)             │               2 │ add_11[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_5    │ (None, 1)                 │               0 │ layer_normalization_11[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_17 (Dense)              │ (None, 1)                 │               2 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,184 (8.53 KB)

 Trainable params: 2,184 (8.53 KB)

...

 Total params: 2,184 (8.53 KB)

 Trainable params: 2,184 (8.53 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0835 - mae: 0.2316 - val_loss: 0.1713 - val_mae: 0.3855
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0515 - mae: 0.1688 - val_loss: 0.1205 - val_mae: 0.3128
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0326 - mae: 0.1381 - val_loss: 0.0927 - val_mae: 0.2653
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0293 - mae: 0.1373 - val_loss: 0.0799 - val_mae: 0.2405
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0286 - mae: 0.1398 - val_loss: 0.0726 - val_mae: 0.2254
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0271 - mae: 0.1380 - val_loss: 0.0731 - val_mae: 0.2264
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0278 - mae: 0.1419 - val_loss: 0.0715 - val_mae: 0.2231
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0276 - mae: 0.1417 - val_loss: 0.0727 - val_mae: 0.2256
Epoch 9/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

✅ Done with capsicum_Udupi_weekly.csv | MAE=1250.41, RMSE=1519.86, R2=-0.06, MAPE=39.87%, Accuracy=60.13%

📊 Metrics saved to tat_metrics.csv


In [ ]:
#TAT+MQA

In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Multi-Query Attention Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads, key_dim, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []

        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)

        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(MultiQueryAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-MQA Model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle NaNs
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_mqa_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_mqa_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.keras"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-MQA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-MQA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_weekly.csv



Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention         │ (None, 30, 64)            │          17,216 │ input_layer_15[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_30 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention[0][… │
│                               │                           │                 │ input_layer_15[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_30        │ (None, 30, 64)            │             128 │ add_30[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_52 (Dense)              │ (None, 30, 128)           │           8,320 │ layer_normalization_30[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_53 (Dense)              │ (None, 30, 1)             │             129 │ dense_52[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_31 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_30[0]… │
│                               │                           │                 │ dense_53[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_31        │ (None, 30, 64)            │             128 │ add_31[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_15   │ (None, 64)                │               0 │ layer_normalization_31[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_54 (Dense)              │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention_9       │ (None, 30, 64)            │          17,216 │ input_layer_24[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_48 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention_9[0… │
│                               │                           │                 │ input_layer_24[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_48        │ (None, 30, 64)            │             128 │ add_48[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_142 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_48[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_143 (Dense)             │ (None, 30, 1)             │             129 │ dense_142[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_49 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_48[0]… │
│                               │                           │                 │ dense_143[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_49        │ (None, 30, 64)            │             128 │ add_49[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_24   │ (None, 64)                │               0 │ layer_normalization_49[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_144 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.5459 - mae: 0.5817 - val_loss: 0.0287 - val_mae: 0.1451
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0266 - mae: 0.1303 - val_loss: 0.0271 - val_mae: 0.1241
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0168 - mae: 0.0975 - val_loss: 0.0274 - val_mae: 0.1246
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0179 - mae: 0.1059 - val_loss: 0.0264 - val_mae: 0.1230
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0168 - mae: 0.0983 - val_loss: 0.0235 - val_mae: 0.1187
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0165 - mae: 0.0984 - val_loss: 0.0236 - val_mae: 0.1187
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0165 - mae: 0.0997 - val_loss: 0.0230 - val_mae: 0.1182
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0135 - mae: 0.0936 - val_loss: 0.0222 - val_mae: 0.1198
Epoch 9/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - lo

In [ ]:
#TAT+GQA

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Grouped Query Attention Layer
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(GroupedQueryAttention, self).get_config()
        config.update({
            "num_groups": self.num_groups,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-GQA Model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle NaNs
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_gqa_model(input_shape=(look_back,1), num_groups=2)
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_gqa_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.keras"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-GQA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_weekly.csv


Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ grouped_query_attention       │ (None, 30, 64)            │           8,768 │ input_layer_25[0][0]       │
│ (GroupedQueryAttention)       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_50 (Add)                  │ (None, 30, 64)            │               0 │ grouped_query_attention[0… │
│                               │                           │                 │ input_layer_25[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_50        │ (None, 30, 64)            │             128 │ add_50[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_150 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_50[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_151 (Dense)             │ (None, 30, 1)             │             129 │ dense_150[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_51 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_50[0]… │
│                               │                           │                 │ dense_151[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_51        │ (None, 30, 64)            │             128 │ add_51[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_25   │ (None, 64)                │               0 │ layer_normalization_51[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_152 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,538 (68.51 KB)

 Trainable params: 17,538 (68.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_34 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ grouped_query_attention_9     │ (None, 30, 64)            │           8,768 │ input_layer_34[0][0]       │
│ (GroupedQueryAttention)       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_68 (Add)                  │ (None, 30, 64)            │               0 │ grouped_query_attention_9… │
│                               │                           │                 │ input_layer_34[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_68        │ (None, 30, 64)            │             128 │ add_68[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_222 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_68[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_223 (Dense)             │ (None, 30, 1)             │             129 │ dense_222[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_69 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_68[0]… │
│                               │                           │                 │ dense_223[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_69        │ (None, 30, 64)            │             128 │ add_69[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_34   │ (None, 64)                │               0 │ layer_normalization_69[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_224 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,538 (68.51 KB)

 Trainable params: 17,538 (68.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.3797 - mae: 0.4690 - val_loss: 0.0271 - val_mae: 0.1241
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0252 - mae: 0.1286 - val_loss: 0.0228 - val_mae: 0.1238
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0194 - mae: 0.1082 - val_loss: 0.0231 - val_mae: 0.1183
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0184 - mae: 0.1062 - val_loss: 0.0224 - val_mae: 0.1213
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0155 - mae: 0.1008 - val_loss: 0.0240 - val_mae: 0.1194
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0180 - mae: 0.1070 - val_loss: 0.0245 - val_mae: 0.1201
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0177 - mae: 0.1048 - val_loss: 0.0222 - val_mae: 0.1190
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0214 - mae: 0.1107 - val_loss: 0.0316 - val_mae: 0.1320
Epoch 9/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - los

In [ ]:
#TAT+HA

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
output_logs = "tat_ha_output_logs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Local Attention Layer
# -----------------------------
class LocalAttention(layers.Layer):
    def __init__(self, key_dim=64, dropout_rate=0.1, **kwargs):
        super(LocalAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = layers.Dense(key_dim)
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x):
        Q = self.q_dense(x)
        K = self.k_dense(x)
        V = self.v_dense(x)
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, V)
        output = self.dropout(output)
        return output

# -----------------------------
# Hierarchical Attention Model
# -----------------------------
def build_tat_ha_model(input_shape, d_model=64, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    local_attn = LocalAttention(key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    local_attn = layers.LayerNormalization(epsilon=1e-6)(local_attn + inputs)
    global_attn = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(local_attn, local_attn)
    global_attn = layers.Dropout(dropout_rate)(global_attn)
    global_attn = layers.LayerNormalization(epsilon=1e-6)(global_attn + local_attn)
    ff = layers.Dense(ff_dim, activation='relu')(global_attn)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + global_attn)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle missing Average Price
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_ha_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_ha_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual & Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model as .pkl
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_ha_model.pkl"))
        joblib.dump(model, model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_ha_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-HA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-HA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_ha_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_weekly.csv


Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_35 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ local_attention               │ (None, 30, 64)            │             384 │ input_layer_35[0][0]       │
│ (LocalAttention)              │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_70 (Add)                  │ (None, 30, 64)            │               0 │ local_attention[0][0],     │
│                               │                           │                 │ input_layer_35[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_70        │ (None, 30, 64)            │             128 │ add_70[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_15       │ (None, 30, 64)            │          66,368 │ layer_normalization_70[0]… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_70[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_52 (Dropout)          │ (None, 30, 64)            │               0 │ multi_head_attention_15[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_71 (Add)                  │ (None, 30, 64)            │               0 │ dropout_52[0][0],          │
│                               │                           │                 │ layer_normalization_70[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_71        │ (None, 30, 64)            │             128 │ add_71[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_228 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_71[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_229 (Dense)             │ (None, 30, 1)             │             129 │ dense_228[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_72 (Add)                  │ (None, 30, 64)            │               0 │ dense_229[0][0],           │
│                               │                           │                 │ layer_normalization_71[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_72        │ (None, 30, 64)            │             128 │ add_72[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_35   │ (None, 64)                │               0 │ layer_normalization_72[0]… │
│ (GlobalAveragePooling1D)      │                           │               

 Total params: 75,650 (295.51 KB)

 Trainable params: 75,650 (295.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_44 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ local_attention_9             │ (None, 30, 64)            │             384 │ input_layer_44[0][0]       │
│ (LocalAttention)              │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_97 (Add)                  │ (None, 30, 64)            │               0 │ local_attention_9[0][0],   │
│                               │                           │                 │ input_layer_44[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_97        │ (None, 30, 64)            │             128 │ add_97[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_24       │ (None, 30, 64)            │          66,368 │ layer_normalization_97[0]… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_97[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_79 (Dropout)          │ (None, 30, 64)            │               0 │ multi_head_attention_24[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_98 (Add)                  │ (None, 30, 64)            │               0 │ dropout_79[0][0],          │
│                               │                           │                 │ layer_normalization_97[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_98        │ (None, 30, 64)            │             128 │ add_98[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_282 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_98[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_283 (Dense)             │ (None, 30, 1)             │             129 │ dense_282[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_99 (Add)                  │ (None, 30, 64)            │               0 │ dense_283[0][0],           │
│                               │                           │                 │ layer_normalization_98[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_99        │ (None, 30, 64)            │             128 │ add_99[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_44   │ (None, 64)                │               0 │ layer_normalization_99[0]… │
│ (GlobalAveragePooling1D)      │                           │               

 Total params: 75,650 (295.51 KB)

 Trainable params: 75,650 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - loss: 0.3984 - mae: 0.5037 - val_loss: 0.0225 - val_mae: 0.1191
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0253 - mae: 0.1217 - val_loss: 0.0249 - val_mae: 0.1205
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0189 - mae: 0.1067 - val_loss: 0.0357 - val_mae: 0.1407
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0186 - mae: 0.1065 - val_loss: 0.0261 - val_mae: 0.1225
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0205 - mae: 0.1155 - val_loss: 0.0269 - val_mae: 0.1238
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0154 - mae: 0.0959 - val_loss: 0.0257 - val_mae: 0.1218
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0172 - mae: 0.1005 - val_loss: 0.0228 - val_mae: 0.1238
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0162 - mae: 0.1009 - val_loss: 0.0259 - val_mae: 0.1221
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.017

In [9]:
#LSTM

In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Paths
input_folder = "dataset"
output_models = "output_lstm_models"
output_csv = "output_lstm_csv"
output_graphs = "output_lstm_graphs"
output_logs = "output_lstm_logs"
metrics_file = os.path.join(output_csv, "lstm_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# Function to create sequences for LSTM
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Metrics storage
metrics_list = []

# Process each CSV file
seq_length = 5
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")
        
        # Load CSV with multiple date formats support
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill missing Average Price
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Scaling
        scaler = MinMaxScaler()
        prices_scaled = scaler.fit_transform(df['Average Price'].values.reshape(-1, 1))

        # Prepare sequences
        X, y = create_sequences(prices_scaled, seq_length)

        # Train-test split (80%-20%)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build LSTM model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        # Train model
        history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round Actual & Predicted
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions in CSV
        df_pred = df.iloc[seq_length + split:].copy()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred['Average Price'] = y_true_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_lstm_updated.csv"), index=False)

        # Metrics calculation
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])
        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_lstm_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph
        plt.figure(figsize=(12,6))
        plt.plot(df_pred['Date'], df_pred['Average Price'], label='Actual', color='blue')
        plt.plot(df_pred['Date'], df_pred['MA_7'], label='MA 7', color='orange')
        plt.plot(df_pred['Date'], df_pred['MA_30'], label='MA 30', color='green')
        plt.plot(df_pred['Date'], df_pred['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f"LSTM Predictions for {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_graph.png"))
        plt.close()

        # Save training loss graph
        plt.figure(figsize=(8,4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"Training Loss for {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_training_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print("\nAll districts processed successfully!")


========== Processing: capsicum_Bangalore_weekly.csv ==========
Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - loss: 0.1207 - mae: 0.3052 - val_loss: 0.1358 - val_mae: 0.3566
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0401 - mae: 0.1592 - val_loss: 0.0235 - val_mae: 0.1296
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0132 - mae: 0.0813 - val_loss: 0.0136 - val_mae: 0.0927
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0096 - mae: 0.0689 - val_loss: 0.0101 - val_mae: 0.0763
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0064 - mae: 0.0617 - val_loss: 0.0102 - val_mae: 0.0768
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0062 - mae: 0.0575 - val_loss: 0.0096 - val_mae: 0.0738
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0067 - mae: 0.0548 - val_loss: 0.0083 - val_mae: 0.0664
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0061 - mae: 0.0557 - val_loss: 0.0085 - val_mae: 0.0682

In [ ]:
#GRU

In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# Paths
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
output_logs = "gru_output_logs"
metrics_file = os.path.join(output_csv, "gru_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# Function to create dataset sequences
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# Store metrics for all files
metrics_list = []

# Process each CSV file
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")
        
        # Load CSV with multiple date formats support
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill missing Average Price
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Prepare data
        values = df[['Average Price']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        X, y = create_dataset(scaled_values, look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        # Train-test split (80%-20%)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build GRU model
        model = Sequential()
        model.add(GRU(64, return_sequences=True, input_shape=(look_back, 1)))
        model.add(Dropout(0.2))
        model.add(GRU(32))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer="adam", loss="mse", metrics=['mae'])

        # Train model
        history = model.fit(X_train, y_train, epochs=20, batch_size=16,
                            validation_data=(X_test, y_test), verbose=1)

        # Save training log
        log_file = os.path.join(output_logs, f"{district_name}_gru_training.txt")
        with open(log_file, "w") as f:
            f.write("Epoch\tTrain_Loss\tVal_Loss\n")
            for i, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
                f.write(f"{i+1}\t{loss:.6f}\t{val_loss:.6f}\n")

        # Predict
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round Actual & Predicted
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions in CSV
        df_pred = df.iloc[-len(y_true):].copy()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred['Average Price'] = y_true_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_gru_updated.csv"), index=False)

        # Metrics calculation
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])
        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_gru_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph
        plt.figure(figsize=(12,6))
        plt.plot(df_pred['Date'], df_pred['Average Price'], label="Actual", color="blue")
        plt.plot(df_pred['Date'], df_pred['MA_7'], label="MA_7", color="orange")
        plt.plot(df_pred['Date'], df_pred['MA_30'], label="MA_30", color="green")
        plt.plot(df_pred['Date'], df_pred['Predicted'], label="Predicted (GRU)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"GRU Predictions - {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_graph.png"))
        plt.close()

        # Save training loss graph
        plt.figure(figsize=(8,4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"GRU Training Loss - {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 Metrics saved to", metrics_file)
print("\n✅ All districts processed successfully!")


========== Processing: capsicum_Bangalore_weekly.csv ==========
Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 14s 86ms/step - loss: 0.0568 - mae: 0.1762 - val_loss: 0.0099 - val_mae: 0.0749
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0100 - mae: 0.0722 - val_loss: 0.0076 - val_mae: 0.0626
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0078 - mae: 0.0646 - val_loss: 0.0110 - val_mae: 0.0818
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0071 - mae: 0.0618 - val_loss: 0.0075 - val_mae: 0.0642
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0060 - mae: 0.0564 - val_loss: 0.0075 - val_mae: 0.0652
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0074 - mae: 0.0617 - val_loss: 0.0064 - val_mae: 0.0592
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0060 - mae: 0.0550 - val_loss: 0.0063 - val_mae: 0.0591
Epoch 8/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0065 - mae: 0.0556 - val_loss: 0.0062 - val_mae: 0.058